In [8]:
import os
import random
import pandas as pd
import json
import sklearn
import spacy
import numpy as np
import torch
import torch.nn.functional as F
from itertools import islice
from tqdm import tqdm
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
sp = spacy.load("en_core_web_sm")

KeyError: 'PUNCTSIDE_FIN'

In [1]:
filepath = "../hein-bound/speeches_111.txt"
with open(filepath, errors="ignore") as f:
    speech = f.readlines()
    speech = [s.replace("\n", "").replace(" | ", " ").split("|") for s in speech]
df = pd.DataFrame(speech[1:], columns = speech[0]) 

NameError: name 'pd' is not defined

In [15]:
df.head(10)

,speech_id,speech
0,1110000001,The Representativeselect and their guests will...
1,1110000002,As directed by law. the Clerk of the House has...
2,1110000003,The quor closes that 428 Represer have respond...
3,1110000004,Credentials. regular in form. have been receiv...
4,1110000005,The Clerk is in receipt of a letter of resigna...
5,1110000006,Pursuant to law and precedent. the next order ...
6,1110000007,Our democracy renews itself every 2 years as M...
7,1110000008,The Clerk now recognizes the gentleman from In...
8,1110000009,Madam Clerk. as chairman of the Republican Con...
9,1110000010,The names of the Honorable NANCY PELOSI. a Rep...


In [16]:
corpus = df.speech

In [17]:
corpus

0         The Representativeselect and their guests will...
1         As directed by law. the Clerk of the House has...
2         The quor closes that 428 Represer have respond...
3         Credentials. regular in form. have been receiv...
4         The Clerk is in receipt of a letter of resigna...
                                ...                        
179263    Madam Speaker. on rollcall Nos. 662 and 661. I...
179264    Madam Speaker. as I leave Congress as the peop...
179265    Madam Speaker. on rolicall No. 658. I was unav...
179266    Madam Speaker. on rollcall No. 658 my flight w...
179267    Madam Speaker. I wold like to submit the follo...
Name: speech, Length: 179268, dtype: object

In [50]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=.5, ngram_range=(1,3))
vectorizer.fit(corpus)

In [30]:
corpus.iloc[19]

'According to precedent. the Chair will swear in the Memberselect en masse. The Memberselect and Delegateselect and the Resident Commissionerelect rose. and the Speaker administered the oath of office to them as follows: Do you solemnly swear or affirm that you will support and defend the Constitution of the United States against all enemies. foreign and domestic. that you will bear true faith and allegiance to the same. that you take this obligation freely. without any mental reservation or purpose of evasion. and that you will well and faithfully discharge the duties of the office on which you are about to enter. so help you God.'

In [46]:
questions = set([s for s in corpus if s.find('?') != -1])

In [49]:
sorted(questions, key=lambda x: len(x), reverse=False)

['$1?',
 'What?',
 'Extra?',
 'Seven?',
 'Sorry?',
 'Pardon?',
 'Totally?',
 'And then?',
 'Im sorry?',
 'How many?',
 'How much?',
 'Hows that?',
 'What page?',
 'Excuse me?',
 'The value?',
 'Pardon me?',
 'For snakes?',
 'I am sorry?',
 'Was it you?',
 'Will yield?',
 'Your health?',
 'Has to what?',
 'Ten minutes?',
 'How about 1?',
 'For my side?',
 'Senator KYL?',
 'He said what?',
 'On your time?',
 'What happens?',
 '2.000 people?',
 'For how long?',
 'There is not?',
 'Five million?',
 'Do we have to?',
 'Then if I may?',
 'Mr. President?',
 'On whose time?',
 'Including you?',
 'How much time?',
 'May I respond?',
 'The answer is?',
 'Breast cancer?',
 'Yield for what?',
 'I dont. do you?',
 'How many times?',
 'Did I say a.m.?',
 'Supporting pie?',
 'Is that better?',
 'Is it paid for?',
 'They come here?',
 'May I continue?',
 'Is that a "no"?',
 'So its exempted?',
 'Is that unusual?',
 'Would you yield?',
 'Why isnt it law?',
 'Who yields time?',
 'TennCare failed?',
 'Wo

In [18]:
speeches = [s for s in corpus if s.find('?') == -1]

In [19]:
len(speeches)

152689

In [20]:
small_speeches = speeches[:10000]

In [77]:
small_speeches[:100]

['The Representativeselect and their guests will please remain standing and join in the Pledge of Allegiance.',
 'As directed by law. the Clerk of the House has prepared the official roll of the Representativeselect. Certificates of election covering 435 seats in the 111th Congress have been received by the Clerk of the House. and the names of those persons whose credentials show that they were regularly elected as Representatives in accordance with the laws of their respective States or of the United States will be called. The Representativeselect will record their presence by electronic device and their names will be recorded in alphabetical order by State. beginning with the State of Alabama. to determine whether a quorum is present. Representativeselect will have a minimum of 15 minutes to record their presence by electronic device. Representativeselect who have not obtained their voting ID cards may do so now in the Speakers lobby.',
 'The quor closes that 428 Represer have respon

### TF.IDF

In [7]:
def lemmatize(phrase):
    return " ".join([word.lemma_ for word in sp(phrase)])

In [4]:
lemmas = [lemmatize(speech) for speech in small_speeches]
df = pd.DataFrame(data={'context': small_speeches, 'lemmas': lemmas})
df.to_feather("context.feather")

NameError: name 'small_speeches' is not defined

In [3]:
df = pd.read_feather("context.feather")
sm_speeches = df.context
lemmas = df.lemmas

In [6]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=5, max_df=.5, ngram_range=(1,2))
tfidf = vectorizer.fit_transform(lemmas)
len(vectorizer.vocabulary_)

22492

Madam Speaker. let me congratulate you for your reelection as Speaker of the House. It is an honor that you tave served with great distinction and verve. I look forward to more of your continued leadership in the 111th Congress. Mr. Speaker. I rise today in support of H. Res. 5. Adopting the rules for the One Hundred Eleventh Congress. The House Rules Package provides commonsense reforms that will enable Congress to work more efficiently for America. In the 110th Congress. Democrats put forth critical measures to restore integrity and accountability to the House. These reforms were the most sweeping ethics and lobbying reforms since Watergate and has changed the way Congress does business in Washington. 


#### The reforms adopted by the 110th Congress included banning gifts from lobbyists. prohibiting the use of corporate jets. mandating ethics training for all House employees. establishing a new. independent Office of Congressional Ethics. and ensuring transparency for budget earmarks by requiring the full disclosure of earmarks in all bills and conference reports. The Rule Package for the 111th Congress builds upon these reforms to further strengthen the integrity of Congress.

In [10]:
question = "What reforms were adopted by the 110th Congress?"
query = vectorizer.transform([lemmatize(question)])
(query > 0).sum(), vectorizer.inverse_transform(query)

(5, [array(['110th', '110th congress', 'adopt', 'congress', 'reform'],
        dtype='<U30')])

In [43]:
scores = (tfidf * query.T).toarray()
results = (np.flip(np.argsort(scores, axis=0)))
for i in results[:5, 0]:
    print(lemmas[i])
    print()

would not affect trade or transportation of animal for zoo . research facility . or other federally licensed and regulated entity . in the 110th congress . the cpsa receive strong support in the 110th congress from dr. jane goodall . the american veterinary medical association . the association of zoos and aquariums . and the humane society of the united states . -PRON- easily pass the house of representatives . -PRON- look forward to work with -PRON- colleague to advance this bipartisan legislation .

madam speaker . let -PRON- congratulate -PRON- for -PRON- reelection as speaker of the house . -PRON- be an honor that -PRON- tave serve with great distinction and verve . -PRON- look forward to more of -PRON- continue leadership in the 111th congress . mr. speaker . -PRON- rise today in support of h. res . 5 . adopt the rule for the one hundred eleventh congress . the house rules package provide commonsense reform that will enable congress to work more efficiently for america . in the 1

### QA

In [29]:
qa_pipeline = pipeline("question-answering")

In [40]:
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:5, 0] ]
contexts = [ (sm_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]

question_df = pd.DataFrame.from_records([ {
    'question': question,
    'context':  ctx
} for (ctx,s) in contexts ])
question_df.to_feather("question_context.feather")

In [41]:
question_df

,question,context
0,What reforms were adopted by the 110th Congress?,would not affect trade or transportation of an...
1,What reforms were adopted by the 110th Congress?,Madam Speaker. let me congratulate you for you...
2,What reforms were adopted by the 110th Congress?,Mr. President. I have sought recognition today...
3,What reforms were adopted by the 110th Congress?,Mr. President. I am pleased to join with the s...
4,What reforms were adopted by the 110th Congress?,Mr. President. pursuant to rule XLIV of the St...


In [42]:
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
answer_df.head(10)

,score,start,end,answer
3,0.891813,2472,2498,Lobbying and Ethics Reform
1,0.563236,764,793,banning gifts from lobbyists.
4,0.555780,2084,2106,nonpublicland measures
0,0.539387,473,495,bipartisan legislation.
2,0.111105,2464,2487,seconddegree amendment.


In [22]:
def get_short_answers(question, qa_pipeline):
    # vectorize question
    query = vectorizer.transform([lemmatize(question)])
    # compute cosine similarity with set of speeches
    scores = (tfidf * query.T).toarray()
    results = (np.flip(np.argsort(scores, axis=0)))
    
    THRESHOLD = 0.01
    candidate_idxs = [ (i, scores[i]) for i in results[0:5, 0] ]
    contexts = [ (sm_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]

    question_df = pd.DataFrame.from_records([ {
        'question': question,
        'context':  ctx
    } for (ctx,s) in contexts ])
    
    preds = qa_pipeline(question_df.to_dict('records'))
    answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
    return answer_df.head()

In [16]:
get_short_answers("What do Democrats believe about tax increases?", qa_pipeline)

,score,start,end,answer
4,0.774658,1870,1896,a stimulus that will fail.
1,0.587369,449,485,keeps the current tax rate the same.
3,0.364495,1511,1556,put some of these businesses out of business.
0,0.090725,299,350,tax increase that is coming full steam ahead a...
2,0.044862,3297,3319,not your income taxes.


In [23]:
get_short_answers("How much working women make compared to men in the same position in the United States?",
                  qa_pipeline)

,score,start,end,answer
0,0.857778,21,29,1 minute
4,0.511652,279,287,78 cents
2,0.387330,1989,2011,77 cents on the dollar
3,0.290338,290,312,78 cents on the dollar
1,0.000018,355,404,shoulder the responsibility but dont get the pay.


### Text generation

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

generated = tokenizer.encode("What reforms were adopted by the 110th Congress??")
context = torch.tensor([generated])
past = None

for i in range(100):
    output, past = model(context, past=past)
    token = torch.argmax(output[..., -1, :])

    generated += [token.tolist()]
    context = token.unsqueeze(0)

sequence = tokenizer.decode(generated)

print(sequence)

I0613 00:31:05.810559 140734868266432 tokenization_utils.py:1078] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /Users/alexandrabutoi/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
I0613 00:31:05.811711 140734868266432 tokenization_utils.py:1078] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /Users/alexandrabutoi/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
I0613 00:31:06.453675 140734868266432 configuration_utils.py:265] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /Users/alexandrabutoi/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.db13c9bc9c7bdd738ec89e069621d88e0

What reforms were adopted by the 110th Congress??

The first was the "National Defense Authorization Act of 2012," which authorized the Pentagon to spend $1.5 trillion over 10 years to increase the military's budget by $1.5 trillion over 10 years. The bill also included a $1.5 trillion increase in the military's budget for fiscal year 2015.

The second was the "Defense Authorization Act of 2012," which authorized the Pentagon to spend $1.5 trillion over 10 years to increase the military's budget by $


### Doc2vec

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess

In [15]:
processed = sm_speeches.apply(simple_preprocess)

In [16]:
processed.head()

0    [the, and, their, guests, will, please, remain...
1    [as, directed, by, law, the, clerk, of, the, h...
2    [the, quor, closes, that, represer, have, resp...
3    [credentials, regular, in, form, have, been, r...
4    [the, clerk, is, in, receipt, of, letter, of, ...
Name: context, dtype: object

In [65]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(processed)]
model = Doc2Vec(documents, vector_size=512, window=2, min_count=3, workers=4, epochs=10)

I0614 01:23:11.255865 140734835215808 doc2vec.py:1377] collecting all words and their counts
I0614 01:23:11.257635 140734835215808 doc2vec.py:1321] PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
I0614 01:23:11.654741 140734835215808 doc2vec.py:1385] collected 29473 word types and 10000 unique tags from a corpus of 10000 examples and 1300414 words
I0614 01:23:11.656124 140734835215808 word2vec.py:1647] Loading a fresh vocabulary
I0614 01:23:11.717775 140734835215808 word2vec.py:1671] effective_min_count=3 retains 13869 unique words (47% of original 29473, drops 15604)
I0614 01:23:11.719250 140734835215808 word2vec.py:1677] effective_min_count=3 leaves 1281069 word corpus (98% of original 1300414, drops 19345)
I0614 01:23:11.799559 140734835215808 word2vec.py:1736] deleting the raw counts dictionary of 29473 items
I0614 01:23:11.801453 140734835215808 word2vec.py:1739] sample=0.001 downsamples 40 most-common words
I0614 01:23:11.802803 140734835215808 word2vec.py:

I0614 01:23:42.326781 140734835215808 base_any2vec.py:1305] EPOCH 8 - PROGRESS: at 26.04% examples, 327219 words/s, in_qsize 7, out_qsize 0
I0614 01:23:43.334192 140734835215808 base_any2vec.py:1305] EPOCH 8 - PROGRESS: at 60.75% examples, 333396 words/s, in_qsize 8, out_qsize 0
I0614 01:23:44.342525 140734835215808 base_any2vec.py:1305] EPOCH 8 - PROGRESS: at 96.10% examples, 312104 words/s, in_qsize 5, out_qsize 0
I0614 01:23:44.370332 140734835215808 base_any2vec.py:349] worker thread finished; awaiting finish of 3 more threads
I0614 01:23:44.406764 140734835215808 base_any2vec.py:349] worker thread finished; awaiting finish of 2 more threads
I0614 01:23:44.441722 140734835215808 base_any2vec.py:349] worker thread finished; awaiting finish of 1 more threads
I0614 01:23:44.445732 140734835215808 base_any2vec.py:349] worker thread finished; awaiting finish of 0 more threads
I0614 01:23:44.447315 140734835215808 base_any2vec.py:1346] EPOCH - 8 : training on 1300414 raw words (979802 ef

In [66]:
qvector = model.infer_vector(simple_preprocess("What do Democrats believe about tax increases?"))

In [67]:
sims = model.docvecs.most_similar([qvector], topn=10)

I0614 01:23:57.182243 140734835215808 keyedvectors.py:1646] precomputing L2-norms of doc weight vectors


In [68]:
sims

[(1897, 0.6019690036773682),
 (9776, 0.5725351572036743),
 (9072, 0.5456220507621765),
 (9303, 0.5355871915817261),
 (5239, 0.5288966298103333),
 (6590, 0.5202503800392151),
 (9162, 0.5160074234008789),
 (3155, 0.5117397904396057),
 (7916, 0.5084536671638489),
 (8309, 0.4982852041721344)]

In [69]:
docs = [i for i, j in sims]

In [70]:
x = sm_speeches[docs]

In [71]:
x

1897    Madam Speaker. this is a sad day. Its a sad da...
9776    Mr. President. I want to speak for a moment ab...
9072    Mr. President. reserving the right to object. ...
9303    Mr. President. the next vote will be on Monday...
5239    proposes an amendment planning. This policy is...
6590          Mr. President. I need about 7 or 8 minutes.
9162    Madam President. reserving the right to object...
3155    Madam Speaker. much is being said about the pr...
7916    Mr. President. I would like to respond to some...
8309    Mr. President. I will put my ability to speak ...
Name: context, dtype: object

In [72]:
question_df = pd.DataFrame.from_records([ {
    'question': "What do Democrats believe about tax increases?",
    'context':  ctx
} for ctx in x ])
question_df

,question,context
0,What do Democrats believe about tax increases?,Madam Speaker. this is a sad day. Its a sad da...
1,What do Democrats believe about tax increases?,Mr. President. I want to speak for a moment ab...
2,What do Democrats believe about tax increases?,Mr. President. reserving the right to object. ...
3,What do Democrats believe about tax increases?,Mr. President. the next vote will be on Monday...
4,What do Democrats believe about tax increases?,proposes an amendment planning. This policy is...
5,What do Democrats believe about tax increases?,Mr. President. I need about 7 or 8 minutes.
6,What do Democrats believe about tax increases?,Madam President. reserving the right to object...
7,What do Democrats believe about tax increases?,Madam Speaker. much is being said about the pr...
8,What do Democrats believe about tax increases?,Mr. President. I would like to respond to some...
9,What do Democrats believe about tax increases?,Mr. President. I will put my ability to speak ...


In [73]:
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)

In [74]:
answer_df

,score,start,end,answer
2,0.290470,154,162,5 oclock.
0,0.235915,771,811,Republicans dont care about health care.
8,0.202437,42,55,some comments
5,0.178406,15,42,I need about 7 or 8 minutes.
7,0.114170,700,764,The House Democrats bill provides enough spend...
3,0.069440,0,14,Mr. President.
9,0.066318,190,216,policies I disagreed with.
6,0.059903,345,390,what opportunity there will be to get a vote.
4,0.025305,88,110,it is instead about en-
1,0.019844,492,548,it Is not a choice between doing nothing on th...


### Universal Sentence Encoder

In [8]:
import tensorflow_hub as hub
from scipy.spatial.distance import cosine
import hnswlib
from sklearn.metrics.pairwise import cosine_similarity

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)

In [11]:
def embed(input):
  return model(input)

In [15]:
small_speeches[:3]

['The Representativeselect and their guests will please remain standing and join in the Pledge of Allegiance.',
 'As directed by law. the Clerk of the House has prepared the official roll of the Representativeselect. Certificates of election covering 435 seats in the 111th Congress have been received by the Clerk of the House. and the names of those persons whose credentials show that they were regularly elected as Representatives in accordance with the laws of their respective States or of the United States will be called. The Representativeselect will record their presence by electronic device and their names will be recorded in alphabetical order by State. beginning with the State of Alabama. to determine whether a quorum is present. Representativeselect will have a minimum of 15 minutes to record their presence by electronic device. Representativeselect who have not obtained their voting ID cards may do so now in the Speakers lobby.',
 'The quor closes that 428 Represer have respon

In [21]:
embeded = embed(small_speeches)

In [22]:
embeded.shape

TensorShape([10000, 512])

In [33]:
query = embed(["What reforms were adopted by the 110th Congress?"])

In [34]:
query.numpy().shape

(1, 512)

In [35]:
scores = cosine_similarity(embeded.numpy(), query.numpy())
results = (np.flip(np.argsort(scores, axis=0)))

In [36]:
results.shape

(10000, 1)

In [37]:
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (small_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]

In [38]:
question_df = pd.DataFrame.from_records([ {
        'question': "What do Democrats believe about tax increases?",
        'context':  ctx
} for (ctx,s) in contexts ])
    
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)

In [39]:
answer_df

,score,start,end,answer
2,0.575306,0,16,Congratulations.
0,0.372675,0,16,Congratulations.
1,0.372675,0,16,Congratulations.
8,0.366115,263,274,opposed it.
3,0.302926,478,569,Democrats put forth critical measures to resto...
5,0.146672,501,575,Democratic majority is severely restricting op...
9,0.037741,430,471,Democratic Caucus from the Presidentelect
6,0.018020,1427,1481,every taxpayer should be up in arms over these...
4,0.005402,129,168,The bill was passed. as follows: S. 181
7,0.000319,1622,1654,I reserve the balance of my time.


In [41]:
query = embed(["What do Democrats believe about tax increases?"])
scores = cosine_similarity(embeded.numpy(), query.numpy())
results = (np.flip(np.argsort(scores, axis=0)))
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (small_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]
question_df = pd.DataFrame.from_records([ {
        'question': "What do Democrats believe about tax increases?",
        'context':  ctx
} for (ctx,s) in contexts ])
    
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
answer_df

,score,start,end,answer
6,0.587369,449,485,keeps the current tax rate the same.
9,0.407120,51,126,Republicans are no less interested in doing th...
5,0.339941,94,146,Those who pay income taxes will get a tax redu...
4,0.331726,1016,1039,irresponsible spending.
3,0.127949,134,162,My colleagues love tax cuts.
7,0.092917,546,604,spent even more money than the new money that ...
1,0.090725,299,350,tax increase that is coming full steam ahead a...
0,0.090094,917,975,average taxpayers saw their earnings decline a...
8,0.038465,806,841,it eliminates the child tax credit.
2,0.019014,262,285,we have the right idea.


In [42]:
query = embed(["How much working women make compared to men in the same position in the United States?"])
scores = cosine_similarity(embeded.numpy(), query.numpy())
results = (np.flip(np.argsort(scores, axis=0)))
THRESHOLD = 0.01
candidate_idxs = [ (i, scores[i]) for i in results[0:10, 0] ]
contexts = [ (small_speeches[i],s) for (i,s) in candidate_idxs if s > THRESHOLD ]
question_df = pd.DataFrame.from_records([ {
        'question': "What do Democrats believe about tax increases?",
        'context':  ctx
} for (ctx,s) in contexts ])
    
preds = qa_pipeline(question_df.to_dict('records'))
answer_df = pd.DataFrame.from_records(preds).sort_values(by="score", ascending=False)
answer_df

,score,start,end,answer
4,6.225415e-02,845,897,Please support equal pay for equal work and vo...
7,1.866715e-02,87,142,women receive equal opportunity to be trained ...
1,1.599180e-02,452,465,family issue.
3,7.326970e-03,1110,1160,this statistic is unlikely to change anytime s...
5,3.693898e-04,1116,1157,families are forced to make do with less.
8,2.004129e-04,398,477,we wouldnt need legislation to reinforce a con...
2,3.591713e-06,1188,1245,closing the pay gap in Michigan and throughout...
0,3.383946e-06,1046,1073,growing economic recession.
9,1.623113e-06,678,728,women have reached full equality in the workpl...
6,6.636560e-07,1539,1544,1963.


In [43]:
contexts

[('Mr. Speaker. I rise today in support of two critical bills: the Lilly Ledbetter Fair Pay Act and the Paycheck Fairness Act. More than 40 years have passed since the passage of the Equal Pay Act. Unfortunately. women still earn less than men for the same work. and that is unfair. Women earn 78 cents for every dollar earned by men. The pay gap is even more severe amongst minority women. African American women earn 69 cents. Hispanic women earn just 56 cents on the dollar. The story of Lilly Ledbetter is a case in point of the barriers faced by women today. Its time for a change. Simply because of her gender. Ms. Ledbetter was paid 20 percent less for performing the same job that her male colleagues performed. Its unfair. The gap doesnt just affect women. It affects the Nations economy. Women represent 41 percent of the total heads of households and sole income earners in this country. We cannot afford to weaken the ability of our breadwinners to pay for the basics. like groceries. chi

### Coreference Resolution & Open Information Extraction

In [1]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.coref

ModuleNotFoundError: No module named 'allennlp'

In [ ]:
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")
predictor.predict(
  document="The woman reading a newspaper sat on the bench with her dog."
)